In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yaml


Crear el DataFrame con cada hora del día (0 a 23)


In [2]:
# Genera el rango de fechas en bloques más pequeños si es necesario
fechas_2023 = pd.date_range(start="2023-01-01", end="2023-12-31", freq="D")
fechas_2024 = pd.date_range(start="2024-01-01", end="2024-12-31", freq="D")
fechas = fechas_2023.append(fechas_2024)

# O usa un enfoque más eficiente para crear las columnas
dim_tiempo = pd.DataFrame({
    "tiempo_key": range(1, len(fechas)+1),  # PK incremental
    "Año": fechas.year,
    "Mes": fechas.month,
    "Dia": fechas.day,
    #"Hora": fechas.hour,
    "Semana": fechas.isocalendar().week,
    "Trimestre": fechas.quarter
})

dim_tiempo

,tiempo_key,Año,Mes,Dia,Semana,Trimestre
2023-01-01,1,2023,1,1,52,1
2023-01-02,2,2023,1,2,1,1
2023-01-03,3,2023,1,3,1,1
2023-01-04,4,2023,1,4,1,1
2023-01-05,5,2023,1,5,1,1
...,...,...,...,...,...,...
2024-12-27,727,2024,12,27,52,4
2024-12-28,728,2024,12,28,52,4
2024-12-29,729,2024,12,29,52,4
2024-12-30,730,2024,12,30,1,4


Añadir una columna para el ID único de cada hora


In [3]:
dim_tiempo["tiempo_key"] = dim_tiempo.index


Reorganizar columnas para que Tiempo_id sea la primera


In [4]:
dim_tiempo = dim_tiempo[["tiempo_key", "Año", "Mes", "Dia"]]

Cargar configuración de la base de datos desde el archivo YAML


In [5]:
with open('../../configBD/config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_etl = config['bodega']

Construir la URL de conexión a la base de datos


In [6]:
url_etl = (f"{config_etl['driver']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['db']}")

Crear el motor de conexión a la base de datos


In [7]:
cliente_etl = create_engine(url_etl)

Guardar el DataFrame en la base de datos como la tabla dim_tiempo


In [8]:
dim_tiempo.to_sql('dim_tiempo', cliente_etl, if_exists='replace', index=False, index_label='Tiempo_id')


731

In [9]:
dim_tiempo

,tiempo_key,Año,Mes,Dia
2023-01-01,2023-01-01,2023,1,1
2023-01-02,2023-01-02,2023,1,2
2023-01-03,2023-01-03,2023,1,3
2023-01-04,2023-01-04,2023,1,4
2023-01-05,2023-01-05,2023,1,5
...,...,...,...,...
2024-12-27,2024-12-27,2024,12,27
2024-12-28,2024-12-28,2024,12,28
2024-12-29,2024-12-29,2024,12,29
2024-12-30,2024-12-30,2024,12,30
